---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    state_univ = []
    curr_state = ''
    with open('university_towns.txt') as f:
        for line in f:
            if '[edit]' in line:
                curr_state = line[:line.index('[')] 
            else:
                if ':' in line:
                    continue
                if "(" in line:
                    state_univ.append((curr_state ,line[:line.index(' (')]))
                else:
                    state_univ.append((curr_state, line.split(',')[0]))
    return pd.DataFrame.from_records(state_univ, columns=["State", "RegionName"])

get_list_of_university_towns().head(15)

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
gdp = pd.read_excel('gdplev.xls', skiprows=7, usecols="E,F,G")
gdp.columns = ['Year_QT', 'GDP_Billions', 'GDP_Chained_2009']
gdp = gdp.loc[(gdp['Year_QT'] == '2000q1').idxmax():] 
gdp.head(15)

,Year_QT,GDP_Billions,GDP_Chained_2009
212,2000q1,10031.0,12359.1
213,2000q2,10278.3,12592.5
214,2000q3,10357.4,12607.7
215,2000q4,10472.3,12679.3
216,2001q1,10508.1,12643.3
217,2001q2,10638.4,12710.3
218,2001q3,10639.5,12670.1
219,2001q4,10701.3,12705.3
220,2002q1,10834.4,12822.3
221,2002q2,10934.8,12893.0


In [5]:
def get_recession_start():
    
    def is_recession_start(w):
        return ((w[0] > w[1]) 
            and (w[1] > w[2]))

    return gdp.loc[(gdp.loc[gdp['GDP_Billions'].rolling(3).apply(is_recession_start) == 1].index - 2).tolist()[0]]['Year_QT']

get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    
    # GDP Recession
    gdpr = gdp.loc[(gdp['Year_QT'] == get_recession_start()).idxmax():]

    def is_recession_end(w):
        return ((w[0] < w[1]) 
            and (w[1] < w[2]))

    return gdpr.loc[(gdpr.loc[gdpr['GDP_Billions'].rolling(3).apply(is_recession_end) == 1].index - 2).tolist()[-1]]['Year_QT']

get_recession_end()

'2015q4'

In [7]:
def get_recession_bottom():
    rec_start = get_recession_start()
    rec_end = get_recession_end()
    rec_start_loc = (gdp['Year_QT'] == rec_start).idxmax()
    rec_end_loc = (gdp['Year_QT'] == rec_end).idxmax()
    min_loc = gdp.loc[rec_start_loc:rec_end_loc]['GDP_Billions'].idxmin()
    return gdp.loc[min_loc]['Year_QT']
    
    
get_recession_bottom()

'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return "ANSWER"

In [9]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"